In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

2023-04-11 04:51:51.895498: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-11 04:51:51.895528: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Import the data

In [2]:
training = pd.read_csv("training over and under 3.csv")
test = pd.read_csv("val3.csv")

training = training[["Peruntukan","Pusat_kota","Visibilitas","Bangunan","Luas"]]
test = test[["Peruntukan","Pusat_kota","Visibilitas","Bangunan","Luas"]]

# Change categorical to number 

In [3]:
condition = [  training.Pusat_kota == "Pusat",
             training.Pusat_kota == "Sedang",
              training.Pusat_kota == "Pinggir",
]

value = [3,2,1]

training.Pusat_kota = np.select(condition,value)
####
####

condition = [  training.Visibilitas == "Strategis",
             training.Visibilitas == "Sedang",
              training.Visibilitas == "Kurang",
]

value = [3,2,1]

training.Visibilitas = np.select(condition,value)
#####
#####

condition = [  training.Bangunan == "Bagus",
             training.Bangunan == "Sedang",
             
]

value = [2,1]

training.Bangunan = np.select(condition,value,0)


condition = [ 
            training.Peruntukan == "Pasar",
             training.Peruntukan == "Kantor",
             training.Peruntukan == "Ruko",
             training.Peruntukan == "Taman",
             training.Peruntukan == "Perumahan",
             training.Peruntukan == "Sawah",
             
]

value = [5,4,3,2,1,0]

training.Peruntukan = np.select(condition,value,0)

In [4]:
condition = [  test.Pusat_kota == "Pusat",
             test.Pusat_kota == "Sedang",
              test.Pusat_kota == "Pinggir",
]

value = [3,2,1]

test.Pusat_kota = np.select(condition,value)
####
####

condition = [  test.Visibilitas == "Strategis",
             test.Visibilitas == "Sedang",
              test.Visibilitas == "Kurang",
]

value = [3,2,1]

test.Visibilitas = np.select(condition,value)
#####
#####

condition = [  test.Bangunan == "Bagus",
             test.Bangunan == "Sedang",
             
]

value = [2,1]

test.Bangunan = np.select(condition,value,0)


condition = [ 
            test.Peruntukan == "Pasar",
             test.Peruntukan == "Kantor",
             test.Peruntukan == "Ruko",
             test.Peruntukan == "Taman",
             test.Peruntukan == "Perumahan",
             test.Peruntukan == "Sawah",
             
]

value = [5,4,3,2,1,0]

test.Peruntukan = np.select(condition,value,0)

In [5]:
training

,Peruntukan,Pusat_kota,Visibilitas,Bangunan,Luas
0,1,2,3,1,87580
1,1,2,3,1,87580
2,1,2,3,1,87580
3,1,2,3,1,87580
4,1,2,3,1,57945
...,...,...,...,...,...
123,2,3,3,0,1624
124,2,3,3,0,17033
125,2,3,3,0,2074
126,2,3,3,0,1700


In [6]:
test

,Peruntukan,Pusat_kota,Visibilitas,Bangunan,Luas
0,4,3,3,2,1192
1,4,3,2,2,997
2,5,3,3,2,14060
3,5,3,3,1,6025
4,0,1,1,0,10390
5,0,1,1,0,7745
6,0,1,1,0,5700
7,1,2,3,1,27024
8,1,1,2,1,8740
9,3,3,3,2,3614


# Dataframe to Dataset

In [7]:
def df_to_dataset(dataframe, shuffle=True, batch_size=4):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Peruntukan')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [8]:
train_ds = df_to_dataset(training)
val_ds = df_to_dataset(test)

2023-04-11 04:51:55.758027: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-11 04:51:55.758072: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-11 04:51:55.758098: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gusanwa): /proc/driver/nvidia/version does not exist
2023-04-11 04:51:55.758429: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Feature Normalizer

In [9]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [10]:
all_inputs = []
encoded_features = []

# Numerical features.
for header in ["Pusat_kota","Visibilitas","Bangunan","Luas"]:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

# Model build

In [15]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(6)(x)

model = tf.keras.Model(all_inputs, output)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [16]:
# Use `rankdir='LR'` to make the graph horizontal.
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [17]:
model.fit(train_ds, epochs=100, validation_data=val_ds)

Epoch 1/100
32/32 [==============================] - 1s 8ms/step - loss: 1.5841 - accuracy: 0.3672 - val_loss: 1.4990 - val_accuracy: 0.5385
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: 1.4865 - accuracy: 0.4062 - val_loss: 1.3911 - val_accuracy: 0.5385
Epoch 3/100
32/32 [==============================] - 0s 3ms/step - loss: 1.4350 - accuracy: 0.4297 - val_loss: 1.2988 - val_accuracy: 0.6923
Epoch 4/100
32/32 [==============================] - 0s 3ms/step - loss: 1.3430 - accuracy: 0.5469 - val_loss: 1.2167 - val_accuracy: 0.6923
Epoch 5/100
32/32 [==============================] - 0s 3ms/step - loss: 1.2499 - accuracy: 0.5938 - val_loss: 1.1479 - val_accuracy: 0.6923
Epoch 6/100
32/32 [==============================] - 0s 3ms/step - loss: 1.2059 - accuracy: 0.6016 - val_loss: 1.0873 - val_accuracy: 0.6923
Epoch 7/100
32/32 [==============================] - 0s 3ms/step - loss: 1.2209 - accuracy: 0.6094 - val_loss: 1.0293 - val_accuracy: 0.6923
Epoch 8/100
3

Epoch 59/100
32/32 [==============================] - 0s 3ms/step - loss: 0.3502 - accuracy: 0.8906 - val_loss: 0.3254 - val_accuracy: 0.9231
Epoch 60/100
32/32 [==============================] - 0s 3ms/step - loss: 0.4088 - accuracy: 0.8594 - val_loss: 0.3193 - val_accuracy: 0.9231
Epoch 61/100
32/32 [==============================] - 0s 3ms/step - loss: 0.3858 - accuracy: 0.9062 - val_loss: 0.3205 - val_accuracy: 0.9231
Epoch 62/100
32/32 [==============================] - 0s 3ms/step - loss: 0.4482 - accuracy: 0.8359 - val_loss: 0.3183 - val_accuracy: 0.9231
Epoch 63/100
32/32 [==============================] - 0s 3ms/step - loss: 0.3622 - accuracy: 0.8906 - val_loss: 0.3132 - val_accuracy: 0.9231
Epoch 64/100
32/32 [==============================] - 0s 3ms/step - loss: 0.4021 - accuracy: 0.8594 - val_loss: 0.3081 - val_accuracy: 0.9231
Epoch 65/100
32/32 [==============================] - 0s 3ms/step - loss: 0.3861 - accuracy: 0.9062 - val_loss: 0.3069 - val_accuracy: 0.9231
Epoch 

In [18]:
model.predict(val_ds)

array([[  8.557469  ,   2.4833074 ,  -3.789686  ,  -7.67444   ,
        -11.909137  , -16.187918  ],
       [-10.712209  ,  -4.9064255 ,  -8.377831  ,   2.9894655 ,
          1.5022533 ,   0.17066216],
       [ -3.217384  ,   3.4401789 ,  -2.5469477 ,  -7.164917  ,
         -5.590574  ,  -2.2140064 ],
       [  8.445325  ,   2.4447198 ,  -3.9958322 ,  -7.248811  ,
        -11.845001  , -16.350441  ],
       [  8.494224  ,   2.4615455 ,  -3.905945  ,  -7.4343996 ,
        -11.872966  , -16.279577  ],
       [ -3.076512  ,  -0.32900867,  -3.815641  ,   2.141334  ,
         -1.5583131 ,  -2.815093  ],
       [-10.657987  ,  -4.856979  ,  -8.302054  ,   2.935988  ,
          1.460021  ,   0.21036813],
       [ -4.0244184 ,  -0.81357193,   5.0505137 ,  -4.2108097 ,
        -10.614844  ,  -1.5301974 ],
       [ -7.1583953 ,  -2.2468872 ,  -1.3000637 ,  -0.25774807,
         -3.2599876 ,   2.1619375 ],
       [  1.0366547 ,   4.858535  ,  -7.411042  ,  -8.199032  ,
         -2.471248  ,  -8.4

In [14]:
all_features = tf.keras.layers.concatenate(encoded_features)


model = tf.keras.Sequential([
  normalization_layer,
  layers.Dense(16, activation='relu'),
  layers.Dense(32, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(6)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=100)

ValueError: Input 0 of layer "dense_2" is incompatible with the layer: expected min_ndim=2, found ndim=1. Full shape received: (None,)